# This Jupyter Notebook goes through a mock email-delivery system involving a given user's invoice.

## The main goal is to take a ```.csv``` file containing transactions and to send an email comprising its summary. 

## The transactions files are structured in the following fashion:
### An _ID_ column to differentiate each transaction; a _DATE_ column which, for simplicity sake, will only contain Day/Month; and a _TRANSACTION_  column with movements, which will differentiate credit movements (+) from debit movements (-). 
## For instance, the following image displays the pandas DataFrame of the ```trxn.csv``` file.
![transactions](images/imagestrxn.png)
## The email should include the following information:
### Total Balance(Credit - Debit movements), Average Credit, Average Debit and the number of transactions per month. 
### As well, a brief preamble, a signature with a logo and a subject must be included.
## The email from the previous ```trxn.csv``` file should look something like this:
![email](images/imagesemail_1try.png)


## CSV file managment

### Using pandas and its built-in datetime utilities, the ```.csv``` files can be handled and processed easily. 

In [15]:
import pandas as pd

def csv2Summary(trxns_file):
    '''
        input =>    
            trxns_file: String
        output =>   
            trxns: Pandas DataFrame
            total_balance: Float
            avg_credit: Float
            avg_debit: Float
            monthly_sumarry: String
                
    '''
    #Total Balance
    trxns = pd.read_csv("trxn.csv")
    total_balance = trxns.transaction.sum() # sum of transactions
    #Debit and Credit Averages
    trxns["DebitorCredit"] = trxns.transaction.map( lambda x: 'credit' if x > 0 else 'debit')
    avg_credit = trxns.loc[trxns.DebitorCredit == "credit", 'transaction'].mean()
    avg_debit = trxns.loc[trxns.DebitorCredit == "debit", 'transaction'].mean()
    #Monthly Summaries
    trxns.date = pd.to_datetime(trxns.date, format = "%d/%m")
    trxns['month'] = trxns.date.dt.strftime('%B')
    trx_per_month = trxns.month.value_counts()
    monthly_summary = ""
    #Create an html-formatted string to inject into the email body for only the months included
    for month, count in zip(trx_per_month.index, trx_per_month):
        monthly_summary += f"<br>Number of transactions in {month}: {count}</br>\n"
    
    return trxns, total_balance, avg_credit, avg_debit, monthly_summary

## Pythonic Email Delivery

### Using smtplib, SSL and Gmail, emails will be sent to a given account
### The authenticaton credentials for the sender will be stored in a pickle file, which only has to be created once. 

## Pickle File for authentication credentials

### Fill the credentials dictionary, run the code once, and then erase the entries.

### This code will create a ```.pkl``` file which will contain the mailing credentials. If the file already exists, then it will load it. 

### This way, sharing code is possible without revealing personal credentials;  ```.gitignore``` already includes this ```.pkl``` file, so it's not passed onto the GitHUB repo. 

In [16]:
import pickle
import os
if not os.path.exists('secret_credentials.pkl'):
    credentials={}
    credentials['Sender Email'] = "" # sender email account
    credentials['Sender Password'] = "" # sender email account password
    credentials['Receiver Email'] = "" # receiver email account
    with open('secret_credentials.pkl','wb') as f:
        pickle.dump(credentials, f)
else:
    credentials=pickle.load(open('secret_credentials.pkl','rb'))

In [17]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

def createTrxnsSummaryMessage(total_balance, avg_credit, avg_debit, monthly_summary):
    '''
        Input =>
            total_balance: Float
            avg_credit: Float
            avg_debit: Float
            monthly_summary: String
        Output=>
            message => SMTP.MIMEMultipart Message Object
    '''
    
    message = MIMEMultipart("alternative")

    
    html = f"""
    <html>
        <body>
            <div>
                <p>
                    Hello,<br>Herein is your Transactions Summary<br>
                </p>
            </div>
            <div>
                <table style="width:100%">
                      <tr>
                        <th>
                            <p>
                                <br>Total Balance: {total_balance}</br>
                                <br>Average Debit Amount: {avg_debit}</br>
                                <br>Average Credit Amount: {avg_credit}</br>
                            </p>
                        </th>
                        <th>
                            <p>
                                {monthly_summary}
                            </p>
                        </th>
                      </tr>
                </table>
            </div>  
            <div>
                <p>
                    <br>Engr. Raúl Armando Murga Garrido</br>
                    <br>B. Sc. Mechatronics Engineering</br>
                    <br><a href="https://www.linkedin.com/in/ra%C3%BAl-murga/">LinkedIn</a></br>
                    <br><a href="https://github.com/ramg93">GitHub</a></br>
                    <br><img align="left" width="150" height="150" src="cid:image1"></br>
                </p>
            </div>
        </body>

    </html>
    """
    html_text = MIMEText(html, "html")
    message.attach(html_text)

    fp = open('images/imageslogo_knot.png', 'rb')
    msgImage = MIMEImage(fp.read())
    fp.close()

    msgImage.add_header('Content-ID', '<image1>')
    message.attach(msgImage)
    
    return message

def sendEmailSLL(credentials, receiver_email, subject, message):
    '''
        Input =>
            credentials: Dict
            receiver_email: String
            subject: String
            message: String}
        Output =>
            None: print either success or exception
    '''
    
    sender_email = credentials['Sender Email']
    password =credentials['Sender Password']
    
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = receiver_email
    
    try:
        port = 465 # Assign 465 port for the SMTP server (Gmail's requirements)
        context = ssl.create_default_context() 

        with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
            server.login(sender_email, password)
            server.sendmail(
                sender_email, 
                receiver_email, 
                message.as_string()
            )
        print("email sent successfully")
    except Exception as e:
        print("error sending email: ", e)

In [18]:
trxns, total_balance, avg_credit, avg_debit, monthly_summary = csv2Summary("trxn.csv")
message = createTrxnsSummaryMessage(total_balance, avg_credit, avg_debit, monthly_summary)
sendEmailSLL(credentials, credentials['Receiver Email'], "Transactions Summary", message)

email sent successfully


# Application to extend a full service out of the Email Sender. 

## Now, let's extend the email functionality to an application where more users can be accessed and a better managment of the data is held. 

## Flask will be the base tool to build the framework and SQLAlchemy will be harnessed to bring together Python and SQL

## The following files created with the```%%writefile``` magic command are the basic app structure and the SQL data models.

In [19]:
%%writefile email_sender_app.py
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret_key'
db_name = "esa.db"
app.config['SQLALCHEMY_DATABASE_URI'] = f'sqlite:///{db_name}'

db = SQLAlchemy(app)
migrate = Migrate(app, db)

from routes import *

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

Overwriting email_sender_app.py


In [13]:
%%writefile models.py
from email_sender_app import db
from datetime import datetime

class User(db.Model):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True)
    created_at = db.Column(db.Date, nullable = False, default=datetime.utcnow())
    name = db.Column(db.String(255), nullable = False)
    lastname = db.Column(db.String(255), nullable = False)
    email = db.Column(db.String(255), nullable = False)
    # relationships 
    transactions = db.relationship('Transaction', back_populates='user', cascade="all,delete")# one2many

    def __repr__(self):
        return f'id = {self.id}: {self.name} {self.lastname}, {self.created_at}'


class Transaction(db.Model):
    __tablename__ = 'transaction'
    id = db.Column(db.Integer, primary_key=True)
    created_at = db.Column(db.Date, nullable = False, default=datetime.utcnow())
    transaction = db.Column(db.Integer)
    # relationships
    user_id = db.Column(db.Integer,  db.ForeignKey("user.id"))# many2one
    user = db.relationship("User", back_populates="transactions")
    
    def __repr__(self):
        return f'id = {self.id}: {self.transaction}, {self.created_at}'

Overwriting models.py


In [33]:
%%writefile routes.py
from email_sender_app import app, db
from flask import render_template, url_for, flash, redirect

import models
from models import *
import forms

@app.route('/')
def base():
    return render_template('base.html')

# ***************************************************** Users **************************************************
@app.route('/users')
def users():
    users = models.User.query.all()
    return render_template('users.html', users=users)

@app.route('/adduser', methods=['GET', 'POST'])
def adduser():
    form = forms.AddUserForm()
    if form.validate_on_submit():
        user = models.User(name=form.name.data,
                           lastname=form.lastname.data,
                           email=form.email.data
                                )
        db.session.add(user)
        db.session.commit()
        flash('User added')
        return redirect(url_for('users'))
    return render_template('adduser.html', form=form)

@app.route('/edit_user/<int:user_id>', methods=['GET', 'POST'])
def edit_user(user_id):
    form = forms.AddUserForm()
    user = models.User.query.get(user_id)
    print(user)
    if user:
        if form.validate_on_submit():
            user.name = form.name.data
            user.lastname = form.lastname.data
            user.email = form. email.data

            db.session.commit()
            flash('User updated')
            return redirect(url_for('users'))
        
        form.name.data = user.name
        form.lastname.data = user.lastname
        form.email.data = user.email
        return render_template('edituser.html', form=form, user_id=user_id)
    flash(f'User with id {user_id} does not exit')
    return redirect(url_for('users'))


@app.route('/delete_user/<int:user_id>', methods=['GET', 'POST'])
def delete_user(user_id):
    form = forms.DeleteForm()
    user = models.User.query.get(user_id)
    if user:
        if form.validate_on_submit():
            if form.submit.data:
                db.session.delete(user)
                db.session.commit()
                flash('User deleted')
            return redirect(url_for('users'))
        return render_template('deleteuser.html', form=form, user_id=user_id)
    flash(f'User with id {user_id} does not exit')
    return redirect(url_for('users'))

# ***************************************************** Transactions **************************************************
@app.route('/transactions')
def transactions():
    transactions = models.Transaction.query.all()
    return render_template('transactions.html', transactions=transactions)

@app.route('/addtransaction', methods=['GET', 'POST'])
def addtransaction():
    form = forms.AddTransactionForm()
    if form.validate_on_submit():
        user = db.session.query(models.User).filter(models.User.id == form.user.data)[0]
        transaction = models.Transaction(transaction=form.transaction.data, 
                                user_id=user.id,
                                user=user
                                )
# add code to determine university id
        db.session.add(transaction)
        db.session.commit()
        flash('Transaction added')
        return redirect(url_for('transactions'))
    return render_template('addtransaction.html', form=form)

@app.route('/edit_transaction/<int:transaction_id>', methods=['GET', 'POST'])
def edit_transaction(transaction_id):
    form = forms.AddTransactionForm()
    transaction = models.Transaction.query.get(transaction_id)
    print(transaction)
    if transaction:
        if form.validate_on_submit():
            user = db.session.query(models.User).filter(models.User.id == form.user.data)[0]
            transaction.transaction = form.transaction.data
            transaction.user = user
            transaction.user_id = user.id

            db.session.commit()
            flash('Transaction updated')
            return redirect(url_for('transactions'))
        form.transaction.data = transaction.transaction
        form.user.data = transaction.user.id
        return render_template('edittransaction.html', form=form, transaction_id=transaction_id)
    flash(f'Transaction with id {transaction_id} does not exit')
    return redirect(url_for('transaction'))

@app.route('/delete_transaction/<int:transaction_id>', methods=['GET', 'POST'])
def delete_transaction(transaction_id):
    form = forms.DeleteForm()
    transaction = models.Transaction.query.get(transaction_id)
    if transaction:
        if form.validate_on_submit():
            if form.submit.data:
                db.session.delete(transaction)
                db.session.commit()
                flash('Transaction deleted')
            return redirect(url_for('transactions'))
        return render_template('deletetransaction.html', form=form, transaction_id=transaction_id)
    flash(f'Transaction with id {transaction_id} does not exit')
    return redirect(url_for('transactions'))

Overwriting routes.py


In [15]:
%%writefile forms.py
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, SelectField
from wtforms.validators import DataRequired, Email


class DeleteForm(FlaskForm):
    submit = SubmitField('Delete')


class AddUserForm(FlaskForm):
    name = StringField('Name', validators=[DataRequired()])
    lastname = StringField('Lastname', validators=[DataRequired()])
    email = StringField('email', validators=[DataRequired(), Email()])
    submit = SubmitField('Submit')
    

class AddTransactionForm(FlaskForm):
    transaction = StringField('Transaction', validators=[DataRequired()])
    user = StringField('User ID', validators=[DataRequired()])
    submit = SubmitField('Submit')

Overwriting forms.py


# Users

In [11]:
%%writefile templates/users.html

{% extends "base.html" %}

{% block main %}
  {% for user in users %}
    <div class="card">
      <div class="card-body">
        <h3>{{ user.name }} {{ user.lastname }}</h3>
        <h4> {{ user.email }}</h4>
        <p>Added on {{ user.created_at }}</p>
        <a href="" class="btn btn-secondary">Transactions</a>
        <a href="{{ url_for('edit_user', user_id=user.id) }}" class="btn btn-primary">Edit</a>
        <a href="{{ url_for('delete_user', user_id=user.id) }}" class="btn btn-danger">Delete</a>
      </div>
    </div>
  {% endfor %}
{% endblock %}

Overwriting templates/users.html


In [5]:
%%writefile templates/adduser.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('adduser') }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Add User</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('users') }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
      <div class="col">
        <h5> Name </h5>
        {{ form.name(class="form-control") }}
      </div>
      <div class="col">
        <h5> Lastname </h5>
        {{ form.lastname(class="form-control") }}
      </div>
      <div class="col">
        <h5> Email </h5>
        {{ form.email(class="form-control") }}
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/adduser.html


In [12]:
%%writefile templates/edituser.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('edit_user', user_id=user_id)  }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Add User</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('users') }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
      <div class="col">
        <h5> Name </h5>
        {{ form.name(class="form-control") }}
      </div>
      <div class="col">
        <h5> Lastname </h5>
        {{ form.lastname(class="form-control") }}
      </div>
      <div class="col">
        <h5> Email </h5>
        {{ form.email(class="form-control") }}
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/edituser.html


In [2]:
%%writefile templates/deleteuser.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('delete_user', user_id=user_id) }}" method="post">
    {{ form.csrf_token }}
    <p>Are you sure you want to delete the following user?</p>
    <div class="row">
      <div class="col">
        <h3>{{ user_id }}</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-danger") }}
        <a href="{{ url_for('users') }}" class="btn btn-primary">Cancel</a>
      </div>
    </div>
  </form>
</div>
{% endblock %}


Writing templates/deleteuser.html


# Transactions

In [30]:
%%writefile templates/transactions.html

{% extends "base.html" %}

{% block main %}
  {% for transaction in transactions %}
    <div class="card">
      <div class="card-body">
        <h3>{{ transaction.transaction }}: {{ transaction.user.id }}</h3>
        <p>Added on {{ transaction.created_at }}</p>
        <a href="{{ url_for('edit_transaction', transaction_id=transaction.id) }}" class="btn btn-primary">Edit</a>
        <a href="{{ url_for('delete_transaction', transaction_id=transaction.id) }}" class="btn btn-danger">Delete</a>
      </div>
    </div>
  {% endfor %}
{% endblock %}

Overwriting templates/transactions.html


In [21]:
%%writefile templates/addtransaction.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('addtransaction') }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Add Transaction</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('transactions') }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
      <div class="col">
        <h5> Transaction </h5>
        {{ form.transaction(class="form-control") }}
      </div>
      <div class="col">
        <h5> User </h5>
        {{ form.user(class="form-control") }}
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/addtransaction.html


In [28]:
%%writefile templates/edittransaction.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('edit_transaction', transaction_id=transaction_id) }}" method="post">
    {{ form.csrf_token }}
    <div class="row">
      <div class="col"> 
        <h3>Edit Transaction</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-primary") }}
        <a href="{{ url_for('transactions') }}" class="btn btn-danger">Cancel</a>
      </div>
    </div>
    <div class="row">
        <div class="col">
          <h5> Transaction </h5>
          {{ form.transaction(class="form-control") }}
        </div>
        <div class="col">
          <h5> User ID </h5>
          {{ form.user(class="form-control") }}
        </div>
      </div>
  </form>
</div>
{% endblock %}

Overwriting templates/edittransaction.html


In [27]:
%%writefile templates/deletetransaction.html

{% extends "base.html" %}

{% block main %}
<div class="container-fluid">
  <form action="{{ url_for('delete_transaction', transaction_id=transaction_id) }}" method="post">
    {{ form.csrf_token }}
    <p>Are you sure you want to delete the following transaction?</p>
    <div class="row">
      <div class="col">
        <h3>{{ transaction_id }}</h3>
      </div>
      <div class="col">
        {{ form.submit(class="btn btn-danger") }}
        <a href="{{ url_for('transactions') }}" class="btn btn-primary">Cancel</a>
      </div>
    </div>
  </form>
</div>
{% endblock %}

Overwriting templates/deletetransaction.html


# Base

In [14]:
%%writefile templates/base.html

<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=yes">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">


    <script src="https://code.jquery.com/jquery-3.2.1.slim.min.js" integrity="sha384-KJ3o2DKtIkvYIK3UENzmM7KCkRr/rE9/Qpg6aAZGJwFDMVNA/GpGFF93hXpG5KkN" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.12.9/umd/popper.min.js" integrity="sha384-ApNbgh9B+Y1QKtv3Rn7W3mgPxhU9K/ScQsAP7hUibX39j7fakFPskvXusvfa0b4Q" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/js/bootstrap.min.js" integrity="sha384-JZR6Spejh4U02d8jOt6vLEHfe/JQGiRRSQQxSfFWpi1MquVdAyjUar5+76PVCmYl" crossorigin="anonymous"></script>
    <title>Pythonic Email Sender: Mock Financial Summary System</title>
  </head>
  <body>
    <div class="container-fluid">
      <h1 class="lead"> 
        <a href="{{ url_for('base') }}">Pythonic Email Sender: Mock Financial Summary System</a>
      </h1>
      <nav class="navbar navbar-expand-lg navbar-light bg-light">
        <ul class="navbar-nav">
          <li class="nav-item dropdown">
            <a class="nav-link dropdown-toggle" href="#" id="navbarDropdown" role="button" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">
              Database Records
            </a>
            <div class="dropdown-menu" aria-labelledby="navbarDropdown">
              <a class="dropdown-item" href="{{ url_for('users') }}">Users</a>
              <a class="dropdown-item" href="{{ url_for('transactions') }}">Transactions</a>
            </div>
          </li>
          <li class="nav-item dropdown">
            <a class="nav-link dropdown-toggle" href="#" id="navbarDropdown" role="button" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">
              Add Records
            </a>
            <div class="dropdown-menu" aria-labelledby="navbarDropdown">
              <a class="dropdown-item" href="{{ url_for('adduser') }}">Add User</a>
              <a class="dropdown-item" href="{{ url_for('addtransaction') }}">Add Transaction</a>
            </div>
          </li>
          <li class="nav-item"><a href="" class="nav-link">Search</a></li>
        </ul>
      </nav>
      <br>
      {% with messages = get_flashed_messages() %}
        {% if messages %}
          {% for message in messages %}
          <div class="alert alert-primary">
            {{ message }}
          </div>
          {% endfor %}
        {% endif %}
      {% endwith %}
      <br>
      {% block main %}{% endblock %}
    </div>
  </body>
</html>

Overwriting templates/base.html
